# Building a Company Research Agent with Linkup & Quotient

<a target="_blank" href="https://colab.research.google.com/github/quotient-ai/quotient-cookbooks/blob/main/cookbooks/search/linkup/linkup-quotient-detections.ipynb">
 <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

This cookbook demonstrates how to build a company description generator using [Linkup](https://docs.linkup.so/) while monitoring result accuracy with [Quotient AI](https://www.quotientai.co/).

We'll cover:
- Using Linkup's API to gather comprehensive company details
- Generating company descriptions with validation
- Monitoring result quality and detecting hallucinations with Quotient
- Understanding and improving the accuracy of company information retrieval


In [1]:
# install dependencies
! pip install -qU quotientai linkup-sdk tqdm

## Step 0: Grab your API keys

We'll use API keys from:
 - [Linkup](https://docs.linkup.so/) — get your API key from the [Linkup app](https://app.linkup.so/sign-up)
 - [Quotient AI](https://www.quotientai.co) — get your API key from the [Quotient AI app](https://app.quotientai.co)
 
Both Linkup and Quotient offer generous free tiers to get started.

In [2]:
import os
# Set API keys:
os.environ['LINKUP_API_KEY'] = "linkup_api_key"
os.environ['QUOTIENT_API_KEY'] = "quotient_api_key"


## Step 1: Set up Linkup Search and Quotient Monitoring

We'll configure Linkup's API to perform comprehensive company research. Our search queries will focus on gathering key information about each company, including:
- Main products and services
- Recent developments and achievements
- Company size and market position
- Key technologies and innovations
- Notable partnerships and milestones


In [3]:
from linkup import LinkupClient
from typing import Dict, Any

# Initialize the client
client = LinkupClient(api_key=os.getenv('LINKUP_API_KEY'))

def generate_query(company_name: str, country: str) -> str:
    query = (
            f"Provide a comprehensive overview of {company_name} in {country}. "
            "Focus on: 1) Their main products and services, "
            "2) Recent major developments and achievements, "
            "3) Company size and market position, "
            "4) Key technologies or innovations they're known for."
        )
    return query

def get_company_info(company_name: str, country: str) -> Dict[str, Any]:
    """
    Get detailed information about a company using Linkup's sourcedAnswer.

    Args:
        company_name: Name of the company
        country: Country where the company operates

    Returns:
        Dictionary containing the answer and source documents
    """
    try:
        # Clean input
        company_name = company_name.strip()
        country = country.strip()
        
        # Generate search query
        query = generate_query(company_name, country)

        # Call Linkup API
        response = client.search(
            query=query,
            depth="deep",  # Use deep for more thorough results
            output_type="sourcedAnswer"
        )
        
        # Access attributes directly from the LinkupSourcedAnswer object
        return {
            'answer': response.answer,
            'sources': response.sources
        }
        
    except Exception as e:
        return {
            'error': str(e),
            'company_name': company_name,
            'country': country
        }


Quotient is an intelligent observability platform designed for retrieval-augmented and search-augmented AI systems.

Quotient performs automated detections on two key fronts each time you send it a log:

- **Hallucination:** Identifies statements in the model output that are unsupported by the retrieved documents or that contradict them. This flagging is done at the sentence level and returns a boolean indicator if any part of the answer contains a hallucination.

- **Document Relevance:** Evaluates each retrieved document to determine whether it meaningfully contributed to grounding the answer. Quotient returns relevance labels for all documents, helping gauge retrieval and search quality.
  
These capabilities are enabled automatically when `hallucination_detection=True` is set during logger initialization.

Below, we'll set up the Quotient logger, send each AI-search result for automatic evaluation, and retrieve structured logs and detections:


In [4]:
from quotientai import QuotientAI

# Initialize the Quotient SDK
quotient = QuotientAI()

quotient.logger.init(
    # Name your application or project
    app_name="linkup-company-research",
    # Set the environment (e.g., "dev", "prod", "staging")
    environment="test",
    # Set the sample rate for logging (0-1.0)
    sample_rate=1.0,
    # Enable hallucination detection
    hallucination_detection=True,
    # Set the sample rate for detections (0-1.0)
    hallucination_detection_sample_rate=1.0,
)


## Step 2: Generate Structured Company Descriptions


We'll test our company description generator with a set of well-known technology companies. For each company, we'll provide:
- Company name
- Country of operation

In [5]:
# Example companies to look up
companies = [
    ("Anthropic", "United States"),
    ("Stripe", "United States"),
    ("OpenAI", "United States"),
    ("Microsoft", "United States"),
    ("Google", "United States")
]


Let's generate detailed, structured descriptions for each company using Linkup's API. For each company, we will:
1. Generate a comprehensive search query
2. Use Linkup's structured output feature to extract company information
3. Format the results according to our schema
4. Log the results in Quotient for quality monitoring

The structured output ensures consistent formatting and makes it easy to use the company information in downstream applications.


In [6]:
log_ids = []

for company_name, country in companies:
    print(f"\nLooking up: {company_name} in {country}")
    result = get_company_info(company_name, country)
    
    if 'error' in result:
        print(f"❌ Error: {result['error']}")
        continue
        
    print(f"\n📊 Company Information:\n{result['answer']}")
    
    # Log to Quotient for quality monitoring
    log_id = quotient.log(
        user_query=f"Get information about {company_name} in {country}",
        model_output=result['answer'],
        documents=[str(doc) for doc in result['sources']]
    )
    
    print(f"📝 Logged to Quotient with log_id: {log_id}")
    log_ids.append(log_id)



Looking up: Anthropic in United States

📊 Company Information:
1) Main products and services: Anthropic offers AI systems focused on safety and reliability, including Claude Opus 4, which delivers superior reasoning, human-quality responses, and brand safety. Their AI excels in customer support, coding, and AI agent collaboration, emphasizing honesty and jailbreak resistance. They provide integration of AI capabilities into applications for faster production-grade solutions.

2) Recent major developments and achievements: Claude Opus 4 is a notable recent advancement, outperforming other models in AI agent scenarios and ranking highest in honesty, jailbreak resistance, and brand safety.

3) Company size and market position: Anthropic is a research and AI safety company with a diverse team from backgrounds including NASA, startups, and the armed forces. They are positioned as a leader in AI safety and reliable AI systems.

4) Key technologies or innovations: Their key innovation is bui

### How It Works

When `.log()` is called:

1. **Data ingestion:** The query, model output, and all retrieved document contents are logged to Quotient.

2. **Async detection pipeline:** Quotient runs:
- **Hallucination detection**, labeling the output as hallucinated or not.
- **Document relevance scoring**, marking which retrieved documents helped ground the output 

3. **Result retrieval:** You can poll or fetch detections linked to your `log_id`.

4. **Monitor and troubleshoot in the Quotient app:** Access the [Quotient dashboard](app.quotientai.co) to:
- Monitor your AI system over time
- Review flagged hallucinated sentences
- See which documents were irrelevant
- Compare across tags or environments for deeper insights

For full implementation details, visit the Quotient [docs](https://docs.quotientai.co/).


# Step 4: Review detections in Quotient

You can now view your logs and detections in the [Quotient dashboard](app.quotientai.co), where you can also filter them by tags and environments to identify common failure patterns.

![Quotient AI Dashboard](Quotient_Dashboard.png "Quotient AI Dashboard")

## What You've Built

A production-ready company description generator that:
- Takes company names and countries as input
- Returns comprehensive, well-researched company information
- Provides source attribution for fact verification
- Monitors accuracy through Quotient's hallucination detection
- Verifies information quality with document relevance scoring

You can scale this to monitor production traffic, benchmark retrieval and search performance, or compare different models side by side.


## Understanding Result Quality

When evaluating the company descriptions, pay attention to these metrics:

- **Hallucination Rate**: Should be **< 5%**
  - Higher rates might indicate:
    - Outdated or conflicting company information online
    - Need for more specific company identifiers in queries
    - Issues with source document quality

- **Document Relevance**: Should be **> 75%**
  - Lower scores might suggest:
    - Company name ambiguity (e.g., common names or multiple companies)
    - Need for additional company identifiers (e.g., industry, location)
    - Source documents discussing different companies or topics

- **Schema Completeness**: Monitor how often optional fields (location, companySize, linkedInUrl) are successfully populated
  - Missing fields might indicate:
    - Information not readily available online
    - Need for additional data sources
    - Opportunities for schema refinement


# (Optional) Analyze Quality Metrics

Let's fetch the quality metrics from Quotient to understand:
- How accurate our company descriptions are
- Which sources were most relevant
- Where we might need additional data or refinement

Quotient's detections are available via the SDK using the `log_id` from each company lookup:


In [8]:
hallucination_detections = []
doc_relevancy_detections = []

from tqdm import tqdm

for id in tqdm(log_ids):
    try:
        detection = quotient.poll_for_detection(log_id=id)
        # Add the hallucination detection to the hallucination_detections list
        hallucination_detections.append(detection.has_hallucination)
        # Add the document relevancy detection to the doc_relevancy_detections list
        docs = detection.log_documents
        doc_relevancy_detections.append(sum(1 for doc in docs if doc.get('is_relevant') is True) / len(docs) if docs else None)
    except Exception as e:
        print(f"Error fetching detection for log_id {id}: {e}")

print(f"Number of results: {len(log_ids)}")
print(f"Percentage of hallucinations: {sum(hallucination_detections)/len(hallucination_detections)*100:.2f}%")
print(f"Average percentage of relevant documents: {sum(doc_relevancy_detections)/len(doc_relevancy_detections)*100:.2f}%")


100%|██████████| 5/5 [08:24<00:00, 100.91s/it]

Number of results: 5
Percentage of hallucinations: 80.00%
Average percentage of relevant documents: 39.04%
